In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

os.chdir('./project')

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


## 0. Prepare libraries

In [2]:
!pip install coloredlogs
!pip install docopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=49f26651428470e183961fdfe1c403c7b137d691a60f03597f753c3d3181c1be
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
!pip install keras-beats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [ ]:
import tensorflow as tf
tf.__version__
import tensorflow.keras.optimizers.legacy

In [ ]:
tensorflow.keras.optimizers.legacy.Adam

keras.optimizers.legacy.adam.Adam

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
!python train.py ./dataset/data/GOOG.csv ./dataset/data/GOOG_2019.csv --strategy dqn --episode-count 50 --model-name doubledqn_goog

2023-06-19 15:27:08.447966: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 15:27:09.408322: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-19 15:27:10 8f8a8aaa1e8f root[546690] DEBUG switching to TensorFlow for CPU
2023-06-19 15:27:10.965861: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-19 15:27:10.965913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: 8f8a8aaa1e8f
2023-06-19 15:27:10.965938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: 8f8a8aaa1e8f
2023-06-19 15:27:10.

In [ ]:
!python train.py ./dataset/data/GOOG.csv ./dataset/data/GOOG_2019.csv --strategy t-dqn --episode-count 5 --model-name doubledqn_goog

2023-06-19 09:47:44.276141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/usr/lib/python3.10/inspect.py", line 2421, in _signature_from_callable
AttributeError: 'function' object has no attribute '__signature__'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/drive/MyDrive/Colab Notebooks/project/train.py", line 34, in <module>
    from trading_bot.agent import Agent
  File "/content/drive/MyDrive/Colab Notebooks/project/trading_bot/agent.py", line 6, in <module>
    import tensorflow as tf
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/__init__.py", line 37, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/__init__.py", line 45, in <module>
    from tensorflow.python.feature_column import feature_column_lib as fe

In [ ]:
from trading_bot.agent import Agent

model_name = 'model_GOOG_50'
test_stock = './dataset/data/GOOG_2019.csv'
window_size = 20
debug = True

agent = Agent(window_size, strategy='double-dqn', pretrained=False)

### 2. Load test data

In [ ]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2019-01-02,1045.849976
1,2019-01-03,1016.059998
2,2019-01-04,1070.709961
3,2019-01-07,1068.390015
4,2019-01-08,1076.280029


### 3. Running Eval

In [ ]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2023-05-31 02:47:18 633684638fd9 root[1152] DEBUG switching to TensorFlow for CPU
2023-05-31 02:47:18 633684638fd9 root[1152] DEBUG Buy at: $1045.85


1/1 [==============================] - 0s 68ms/step


2023-05-31 02:47:18 633684638fd9 root[1152] DEBUG Buy at: $1016.06


1/1 [==============================] - 0s 20ms/step


2023-05-31 02:47:18 633684638fd9 root[1152] DEBUG Buy at: $1070.71


...

2023-05-31 02:47:31 633684638fd9 root[1152] DEBUG Buy at: $1343.56


1/1 [==============================] - 0s 20ms/step


2023-05-31 02:47:31 633684638fd9 root[1152] DEBUG Buy at: $1360.40


1/1 [==============================] - 0s 19ms/step


2023-05-31 02:47:31 633684638fd9 root[1152] DEBUG Buy at: $1351.89
2023-05-31 02:47:31 633684638fd9 root[1152] INFO model_GOOG_50: +$1361.74



## 4. Visualize

In [ ]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions

    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)

    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )

    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)

    return chart

In [ ]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

## 5. Other trials

In [ ]:
!python train.py ./dataset/data/AAPL.csv ./dataset/data/AAPL_2018.csv --strategy dqn --episode-count 10 --model-name tdqn_aapl

2023-06-19 15:51:25.756364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 15:51:26.685351: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-19 15:51:28 8f8a8aaa1e8f root[774185] DEBUG switching to TensorFlow for CPU
2023-06-19 15:51:28.292555: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-19 15:51:28.292597: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: 8f8a8aaa1e8f
2023-06-19 15:51:28.292608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: 8f8a8aaa1e8f
2023-06-19 15:51:28.

In [ ]:
!python train.py ./dataset/data/AAPL.csv ./dataset/data/AAPL_2018.csv --strategy double-dqn --episode-count 10 --model-name tdqn_aapl

2023-06-19 17:03:28.408716: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 17:03:29.431203: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-19 17:03:31 8f8a8aaa1e8f root[1125564] DEBUG switching to TensorFlow for CPU
2023-06-19 17:03:31.067208: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-19 17:03:31.067258: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: 8f8a8aaa1e8f
2023-06-19 17:03:31.067271: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: 8f8a8aaa1e8f
2023-06-19 17:03:31

In [ ]:
model_name = 'model_AAPL'
test_stock = './dataset/data/AAPL.csv'
window_size = 10
debug = True

agent = Agent(window_size, reset_every=2000, strategy='double-dqn', pretrained=False)

In [ ]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Close']]
# rename feature column names
df = df.rename(columns={'Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2010-08-12,35.970001
1,2010-08-13,35.585712
2,2010-08-16,35.377144
3,2010-08-17,35.995716
4,2010-08-18,36.152859


In [ ]:
coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2023-05-31 02:53:50 633684638fd9 root[1152] DEBUG switching to TensorFlow for CPU
2023-05-31 02:53:50 633684638fd9 root[1152] DEBUG Buy at: $24.17


1/1 [==============================] - 0s 19ms/step


2023-05-31 02:53:53 633684638fd9 root[1152] DEBUG Buy at: $29.52


1/1 [==============================] - 0s 20ms/step


2023-05-31 02:53:56 633684638fd9 root[1152] DEBUG Buy at: $33.05


...

2023-05-31 02:55:26 633684638fd9 root[1152] DEBUG Sell at: $146.70 | Position: +$57.56


1/1 [==============================] - 0s 21ms/step


2023-05-31 02:55:26 633684638fd9 root[1152] DEBUG Buy at: $147.24


1/1 [==============================] - 0s 22ms/step


2023-05-31 02:55:26 633684638fd9 root[1152] INFO model_AAPL: +$3161.89



In [ ]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

## 5.1 Ours

In [ ]:
!python train.py ./dataset/archive/BTC-2021min.csv ./dataset/archive/BTC-2021min.csv --strategy t-dqn --episode-count 15 --model-name tdqn_btc

2023-06-20 02:11:58.488221: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 02:11:59.495644: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-20 02:12:01 24832551a444 root[4544] DEBUG switching to TensorFlow for CPU
2023-06-20 02:12:01.055301: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-20 02:12:01.055339: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: 24832551a444
2023-06-20 02:12:01.055351: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: 24832551a444
2023-06-20 02:

In [ ]:
model_name = 'model_BTC_2021'
test_stock = './dataset/archive/BTC-2021min.csv'
window_size = 20
debug = True

agent = Agent(window_size, reset_every=2000, strategy='double-dqn"', pretrained=False)

In [ ]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['date', 'close']]
# rename feature column names
df = df.rename(columns={'close': 'actual', 'date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates
print(df.isnull().any())
df.head()

date      False
actual    False
dtype: bool


,date,actual
0,2022-03-01 03:43:00,43046.58
1,2022-03-01 03:41:00,43046.58
2,2022-03-01 03:40:00,43016.03
3,2022-03-01 03:39:00,42999.44
4,2022-03-01 03:38:00,43049.46


In [ ]:
def get_stock_data(stock_file):
    """Reads stock data from csv file
    """
    df = pd.read_csv(stock_file)
    return list(df['close'])

In [ ]:
coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)[:int(len(df)*0.002)]
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2023-05-31 03:03:44 633684638fd9 root[1152] DEBUG switching to TensorFlow for CPU
2023-05-31 03:03:45 633684638fd9 root[1152] DEBUG Buy at: $43046.58


1/1 [==============================] - 0s 76ms/step


2023-05-31 03:03:45 633684638fd9 root[1152] DEBUG Sell at: $43046.58 | Position: +$0.00


1/1 [==============================] - 0s 20ms/step


2023-05-31 03:03:46 633684638fd9 root[1152] DEBUG Buy at: $42983.81


...

2023-05-31 03:04:47 633684638fd9 root[1152] DEBUG Buy at: $38245.61


1/1 [==============================] - 0s 20ms/step


2023-05-31 03:04:47 633684638fd9 root[1152] DEBUG Sell at: $38328.03 | Position: +$82.42


1/1 [==============================] - 0s 19ms/step


2023-05-31 03:04:51 633684638fd9 root[1152] INFO model_BTC_2021: +$242.08



In [ ]:
chart = visualize(df.iloc[:int(len(df)*0.002)], history, title=test_stock)
chart

<ipython-input-17-09db1cc8c2ab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['position'] = position
<ipython-input-17-09db1cc8c2ab>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['action'] = actions


alt.LayerChart(...)